# Toolhouse and OpenAI Integration for System Design and Reporting

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description:

This app demonstrates the integration of OpenAI's language models with Toolhouse's system design and utility tools. It generates detailed reports, executes commands, and emails results based on user inputs, specifically focusing on system design topics. The app is designed for use in Jupyter notebooks, automating tasks like generating code, system design documentation, and sending email reports. It is highly customizable and uses environment variables for secure API key management.

## Step 1: Installing Python Dependencies

This code sets up the environment for a Jupyter notebook:

- `Install Requirements`: The install_requirements() function checks if the required dependencies from requirements.txt are installed. If not, it installs them and retries up to three times if the installation fails.

- `Environment Setup`: The setup_env() function loads environment variables from a .env file using load_dotenv(). It checks if essential environment variables (like OPENAI_API_KEY and TOOLHOUSE_API_KEY) are set and exits if any are missing.

- `Final Setup` : Once the requirements are installed and environment variables are validated, the setup is marked complete with a success message.


In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "TOOLHOUSE_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Setting Up OpenAI and Toolhouse Integration

This script integrates OpenAI with Toolhouse to generate code based on user input, executes it dynamically, and displays the results, enabling seamless interaction between code generation and execution.


In [ ]:
## Example: Just a basic use of Toolhouse x OpenAI

import os
from openai import OpenAI
from toolhouse import Toolhouse

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")

MODEL = "gpt-4o-mini"

messages = [
    {
        "role": "user",
        "content": "Generate FizzBuzz code."
        "Execute it to show me the results up to 10.",
    }
]

response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=th.get_tools()
)

messages += th.run_tools(response)

response = client.chat.completions.create(
    model=MODEL, messages=messages, tools=th.get_tools()
)

print(response.choices[0].message.content)

## Step 3: Integration of OpenAI with Toolhouse for System Design

This code integrates OpenAI’s language model with Toolhouse’s system design tools, enabling iterative refinement of system design tasks through the `llm` function, which dynamically utilizes both platforms for enhanced efficiency and accuracy.


In [34]:
## Example: LLM with Toolhouse
## Include the following tools in the Toolhouse bundle: thp-system-design-assistant
## Sendgrid, Web Scraper, and Memory tools are included in this bundle.

import os
from openai import OpenAI
from toolhouse import Toolhouse
import traceback

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")
MAX_ITERS = 10
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")
openai = OpenAI(api_key=OPENAI_API_KEY)


def llm(
    prompt: str,
    model="gpt-4o-mini",
    system="You are a helpful AI assistant",
    intermediate_output=True,
    verbose=True,
) -> str:
    """Wrapper over Open AI LLM calls with Toolhouse Tools."""
    try:
        if verbose:
            print("System: ", system)
            print("Prompt: ", prompt)
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            tools=th.get_tools("thp-system-design-assistant"),
        )

        tool_calls = response.choices[0].message.tool_calls
        iters = 0
        content = response.choices[0].message.content

        while tool_calls and len(tool_calls) > 0 and iters < MAX_ITERS:
            if verbose or intermediate_output:
                if content:
                    print(f"Cycle {iters}: ", content)
                else:
                    print(f"Cycle {iters}: No content.")
            messages += th.run_tools(response)
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                tools=th.get_tools("thp-system-design-assistant"),
            )
            if verbose or intermediate_output:
                print(f"Generated intermediate response...")
            content = response.choices[0].message.content
            iters += 1
            tool_calls = response.choices[0].message.tool_calls

        if content and (verbose or intermediate_output):
            print(f"Final response: {content}")

        return content
    except Exception as e:
        traceback.print_exc()
        return "Sorry, failed to generate response.\nError: " + str(e)

## Step 4: System Design Report Generation on "Bloom Filters" with OpenAI and Toolhouse

The process starts with selecting the topic "Bloom Filters" and setting up system and prompt messages for the AI to follow. The AI, acting as a system design assistant, generates a comprehensive report on Bloom Filters, including use cases, advantages, and examples. After generating the markdown report, it is converted to HTML, and the email is sent via Toolhouse's SendGrid integration. The entire process is iterative, refining the report with multiple API calls until the content is polished and ready for delivery.

In [ ]:
## Testing simple 'System Design' report emailer use case

topic = "Bloom Filters"
email = "contact.adityapatange@gmail.com"
model = "gpt-4o"

system = f"""
    You are THP System Design AI.
    An intelligent assistant that helps you understand system design concepts.
    You will provide detailed explanations of the topics you are asked to research.
    I will tell you the various things to research. 
    You will generate a detailed markdown report on the topic.
"""

prompt = f"""
    Provide a detailed explanation of {topic}.
    Include the following points:
    - What is {topic}?
    - How does '{topic}' work?
    - What are the use cases of '{topic}'?
    - Advantages and disadvantages of '{topic}'. (If applicable)
    - Industry examples of {topic}. 

    INSTRUCTIONS:
    - Generate a detailed markdown report on {topic}.
    This is for study purposes, so please provide a detailed explanation.
    - Generated a nicely formatted HTML email on your research with title 'The Hackers Playbook System Design: Case Studies on {topic}'.
    - Send the email to {email}
"""

response = llm(
    prompt, model=model, system=system, intermediate_output=True, verbose=True
)

print(response)

## Conclusion

This app serves as a powerful tool for automating **system design research** and **report generation** by leveraging OpenAI’s language model and Toolhouse’s suite of tools. With capabilities like **GPT-4 integration** and **email delivery through SendGrid**, the app efficiently generates detailed and structured reports on complex topics, such as **Bloom Filters**, and sends them directly to the user’s email.

---

### Key Strengths of the App:

1. **Seamless Integration:**  

   Combines the power of OpenAI and Toolhouse to create a robust environment for automated research and report delivery.

2. **Customizable Functionality:**  

   Users can easily modify the topic or focus area, enabling **flexible, on-demand research generation** tailored to specific needs.

3. **Automated Workflow:**  

   From generating detailed markdown reports to delivering formatted HTML emails, the app automates the entire process, saving time and effort.

4. **Iterative Process:**  

   Supports iterative tool calls to refine content until it meets the desired quality and level of detail.

5. **Practical Use Cases:**  

   Perfect for students, professionals, or teams involved in system design. It is useful for:

   - Studying and exploring new concepts.
   - Researching detailed system design topics.
   - Generating professional reports with easy email sharing.



### Final Thoughts:

With its **flexibility**, **ease of use**, and **ability to deliver tailored insights**, this app is an invaluable resource for anyone looking to **explore or present system design concepts** in a structured, professional manner. It not only accelerates learning but also simplifies **knowledge sharing**, making it an efficient tool for study, research, and professional collaboration.


---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

